# Tuning with keras-tuner

In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import pandas as pd
import tensorflow as tf

# Functions

In [2]:
def load_dataset():
    """ Reads dataset csv and returns pandas dataframe """
    
    filepath = "../Resources/charity_data.csv"

    df = pd.read_csv(filepath, encoding="utf-8", low_memory=False)
    
    return df

In [3]:
def clean_dataset(a_df):
    """ Returns deduped, na-dropped, index-reset dataframe """    
    
    a_df = a_df.drop_duplicates()   
        
    a_df = a_df.dropna()
    
    a_df = a_df.drop(columns=["EIN", "NAME"])
        
    a_df = a_df.reset_index(drop=True)
    
    return a_df   

In [4]:
def examine_dataset(a_df):
    """ Provides summary info and visualizations of dataset """
    
    a_df.info()
    
    print("\n\n")
                  
    # Determine the number of unique values in each column.

    for col in a_df.columns:
        if (a_df[col].nunique() > 10):
            print(f"{col}\n\n{a_df[col].value_counts()}\n\n")  

In [5]:
def reduce_cats(a_col, a_cutoff):
    """ Inputs are a series and a cutoff value for 'Other' """
    
    print(f"BEFORE: \n\n{df[a_col].value_counts()}\n\n")

    types_to_replace = (df[a_col].value_counts().loc[lambda x: x < int(a_cutoff)]).keys().tolist()

    for code in types_to_replace:        
        df[a_col] = df[a_col].replace(code, "Other")

    # Check to make sure binning was successful

    print(f"AFTER: \n\n{df[a_col].value_counts()}\n\n")   

In [6]:
def encode_df(a_df):
    """ Returns one-hot encoded dataframe """
    
    categorical_list = a_df.dtypes[a_df.dtypes == "object"].index.tolist() 
    
    print(f"CATEGORIES FOR EACH CATEGORICAL FEATURE ENCODED:\n\n{a_df[categorical_list].nunique()}\n\n")
    
    concat_list = []
    
    for categorical in categorical_list:
        
        concat_list.append(pd.get_dummies(a_df[categorical], prefix=categorical, prefix_sep='_'))        
    
    concat_list.append(a_df["IS_SUCCESSFUL"])

    return pd.concat(concat_list, axis=1)   

In [7]:
def pre_process(a_df):
    """ Make X,y ... train_test_split ... scale, fit and transform """
    
    # Split our preprocessed data into our features and target arrays

    y = enc_df["IS_SUCCESSFUL"].values
    X = enc_df.drop(["IS_SUCCESSFUL"], axis=1).values

    # Split the preprocessed data into a training and testing dataset

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42) 
    
    # Create a StandardScaler instances
    
    scaler = StandardScaler()

    # Fit the StandardScaler

    X_scaler = scaler.fit(X_train)

    # Scale the data

    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    print(f"TRAIN SCALED SHAPE: {X_train_scaled.shape}")
    print(f"TEST SCALED SHAPE: {X_test_scaled.shape}")

    input_dim = X_train_scaled.shape[1]  
    
    return input_dim, X_train_scaled, X_test_scaled, y_train, y_test

In [8]:
def create_model(hp):
    
    # NB: change input_dim as needed!
    
    # Instantiate a Sequential model
    
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers.
    
    activation = hp.Choice('activation', ['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide the number of neurons in first layer and also
    # the activation function. 
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide the number of hidden layers and number of 
    # neurons in each one
    
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    # Define the output layer
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [9]:
def find_best():
    """ Uses keras-tuner to find best model specs """
    
    tuner = kt.Hyperband(
        create_model,
        objective="val_accuracy",
        max_epochs=10,
        hyperband_iterations=2)
    
    # Run the kerastuner search for best hyperparameters

    tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))
    
     # Get best model hyperparameters
 
    best_hyper = tuner.get_best_hyperparameters(1)[0]

    print(best_hyper.values)
    
     # Evaluate best model against full test data
 
    best_model = tuner.get_best_models(2)[0]
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)

    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
     # Summarize the best model
 
    print(best_model.summary())

# Call functions

In [10]:
df = clean_dataset(load_dataset())

examine_dataset(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB



APPLICATION_TYPE

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T

In [11]:
# Bin columns with > 10 unique values

reduce_cats("APPLICATION_TYPE", 500)

reduce_cats("ASK_AMT", 25_000)

reduce_cats("CLASSIFICATION", 1800)

BEFORE: 

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


AFTER: 

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64


BEFORE: 

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64


AFTER: 

5000     25398
Other     8901
Name: ASK_AMT, dtype: int64


BEFORE: 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


AF

In [12]:
df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,Other,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,Other,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,Other,1


In [13]:
enc_df = encode_df(df)

enc_df

CATEGORIES FOR EACH CATEGORICAL FEATURE ENCODED:

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            6
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
ASK_AMT                   2
dtype: int64




,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_Other,IS_SUCCESSFUL
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
2,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
3,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,1,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
34295,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
34296,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
34297,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1


In [14]:
input_dim, X_train_scaled, X_test_scaled, y_train, y_test = pre_process(enc_df)

TRAIN SCALED SHAPE: (25724, 43)
TEST SCALED SHAPE: (8575, 43)


In [15]:
find_best()

Trial 60 Complete [00h 00m 26s]
val_accuracy: 0.7318950295448303

Best val_accuracy So Far: 0.7318950295448303
Total elapsed time: 00h 11m 42s
INFO:tensorflow:Oracle triggered exit
{'activation': 'relu', 'first_units': 9, 'num_layers': 1, 'units_0': 9, 'units_1': 5, 'units_2': 3, 'units_3': 7, 'units_4': 3, 'units_5': 5, 'tuner/epochs': 10, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
268/268 - 1s - loss: 0.5573 - accuracy: 0.7319 - 747ms/epoch - 3ms/step
Loss: 0.557326078414917, Accuracy: 0.7318950295448303
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 396       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 1)             

# Compile, Train and Evaluate the Model

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# Input layer

nn.add(tf.keras.layers.Dense(units=9, input_dim=43, activation="relu"))

# First hidden layer

nn.add(tf.keras.layers.Dense(units=9, activation="relu"))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=9, activation="tanh"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100) 

# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file

nn.save("../Models/nn_optimized.h5")

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 9)                 396       
                                                                 
 dense_17 (Dense)            (None, 9)                 90        
                                                                 
 dense_18 (Dense)            (None, 9)                 90        
                                                                 
 dense_19 (Dense)            (None, 1)                 10        
                                                                 
Total params: 586
Trainable params: 586
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6008 - accuracy: 0.6923
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - acc

804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accuracy: 0.7349
Epoch 73/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5410 - accuracy: 0.7354
Epoch 74/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7353
Epoch 75/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7358
Epoch 76/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5410 - accuracy: 0.7360
Epoch 77/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7347
Epoch 78/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7353
Epoch 79/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5409 - accuracy: 0.7364
Epoch 80/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7362
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 

# Best result: 

### Loss: 0.5544543266296387, Accuracy: 0.7315452098846436